### Installation of Driver (Python DB API)

In [4]:
#pip install pyodbc

### Connecting to MS SQL Server 

In [5]:
import pyodbc

In [7]:
driver = '{ODBC Driver 17 for SQL Server}'
server = 'localhost'
database = 'master'
user = 'sa'
password = 'Ll123456'

In [8]:
conn = pyodbc.connect(DRIVER=driver, SERVER=server, DATABASE=database, UID=user, PWD=password)

In [9]:
def create_server_connection(driver, server, database, user, password):
    conn = None
    try:
        conn = pyodbc.connect(DRIVER = driver, SERVER = server, DATABASE = database, UID = user, PWD = password)
        print("MS SQL Server Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")
    return conn

In [10]:
# conn = create_server_connection(driver, server, database, user, password)

### Creating a New Database

In [11]:
conn.autocommit =True

In [12]:
create_database_query = 'CREATE DATABASE TestDB'

In [13]:
def create_database(conn, create_database_query):
    crs = conn.cursor()
    try:
        crs.execute(create_database_query)
        print("Database created successfully!")
    except Error as err:
        print(f"Error: '{err}'")

In [14]:
create_database(conn, create_database_query)

Database created successfully!


In [20]:
def execute_query(conn, query):
    crs = conn.cursor()
    try:
        crs.execute(query)
        print("Query successful!")
    except Error as err:
        print(f"Error: '{err}'")

In [17]:
query = "USE TestDB"
execute_query(conn, query)

Query successful!


Please Create A table into our TestDB, the columns are: ID, FirstName. LastName.....Primary Key......

In [23]:
query ='CREATE TABLE TestA (\
            ID int IDENTITY(1,1) NOT NULL, \
            FirstName varchar(255) NOT NULL,\
            LastName varchar(255) NOT NULL,\
            PRIMARY KEY(ID))'

In [24]:
execute_query(conn, query)

Query successful!


### Insert

In [26]:
crs = conn.cursor()

In [27]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES ('Bob', 'Marley')")

In [29]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES ('Bruce', 'Lee')")

In [30]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES ('Tom', 'Hanks')")

In [31]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES ('Bart', 'Simpson')")

In [32]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES ('Tom', 'Cat')")

In [33]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES ('Jerry', 'Mouse')")

In [34]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES ('Stefan', 'Müller')")

In [35]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES ('Michael', 'Jackson')")

In [36]:
conn.commit()

In [37]:
conn.autocommit = True

### Update and Delete

In [38]:
crs.execute("DELETE FROM TestA WHERE FirstName=?", 'Stefan')

In [39]:
crs.rowcount

1

In [40]:
crs.execute("DELETE FROM TestA WHERE LastName=?", 'Lee')
print(crs.rowcount, "'rows deleted!")

2 'rows deleted!


### Reading Data

In [41]:
crs.execute('SELECT FirstName, LastName from TestA')
row = crs.fetchone()
row

('Bob', 'Marley')

In [42]:
row.LastName

'Marley'

In [43]:
row.FirstName

'Bob'

In [45]:
crs.execute('SELECT FirstName, LastName from TestA')
while True:
    row = crs.fetchone()
    if not row:
        break
    print(row.FirstName, row.LastName)

Bob Marley
Tom Hanks
Bart Simpson
Tom Cat
Jerry Mouse
Michael Jackson


In [46]:
row_last = crs.fetchone()
row_last

In [47]:
crs.execute('SELECT FirstName, LastName from TestA')
rows = crs.fetchall()
rows

[('Bob', 'Marley'),
 ('Tom', 'Hanks'),
 ('Bart', 'Simpson'),
 ('Tom', 'Cat'),
 ('Jerry', 'Mouse'),
 ('Michael', 'Jackson')]

In [48]:
for row in rows:
    print(row.FirstName, row.LastName)

Bob Marley
Tom Hanks
Bart Simpson
Tom Cat
Jerry Mouse
Michael Jackson


In [49]:
crs.execute('SELECT FirstName, LastName from TestA')
for row in crs:
    print(row.FirstName, row.LastName)

Bob Marley
Tom Hanks
Bart Simpson
Tom Cat
Jerry Mouse
Michael Jackson


In [50]:
for row in crs.execute('SELECT FirstName, LastName from TestA'):
    print(row.FirstName, row.LastName)

Bob Marley
Tom Hanks
Bart Simpson
Tom Cat
Jerry Mouse
Michael Jackson


In [52]:
crs.execute('SELECT FirstName, LastName from TestA WHERE FirstName = ?', 'Jerry')
row = crs.fetchone()
row

('Jerry', 'Mouse')

In [53]:
row.FirstName

'Jerry'

In [54]:
row.LastName

'Mouse'

In [55]:
type(row)

pyodbc.Row

In [57]:
crs.execute('SELECT FirstName, LastName from TestA WHERE FirstName = ? and LastName=?', ['Tom', 'Cat'])
crs.fetchone()

('Tom', 'Cat')

In [58]:
def read_query(conn, query):
    crs = conn.cursor()
    results = None
    crs.execute(query)
    results = crs.fetchall()
    crs.close()
    return results

In [59]:
query = 'SELECT FirstName, LastName from TestA'
execute_query(conn, query)
results = read_query(conn, query)
results

Query successful!


[('Bob', 'Marley'),
 ('Tom', 'Hanks'),
 ('Bart', 'Simpson'),
 ('Tom', 'Cat'),
 ('Jerry', 'Mouse'),
 ('Michael', 'Jackson')]

In [60]:
import pandas as pd

In [61]:
from_db = []
for result in results:
    result = list(result)
    from_db.append(result)
columns=["FirstName", "LastName"]
df = pd.DataFrame(from_db, columns=columns)
df

,FirstName,LastName
0,Bob,Marley
1,Tom,Hanks
2,Bart,Simpson
3,Tom,Cat
4,Jerry,Mouse
5,Michael,Jackson


In [64]:
rows

[('Bob', 'Marley'),
 ('Tom', 'Hanks'),
 ('Bart', 'Simpson'),
 ('Tom', 'Cat'),
 ('Jerry', 'Mouse'),
 ('Michael', 'Jackson')]

In [66]:
row = crs.execute("SELECT COUNT(*) from TestA").fetchone()
row

(6, )

In [67]:
crs.execute('SELECT FirstName, LastName from TestA WHERE FirstName = ? and LastName=?', ['Tom', 'Cat'])
row = crs.fetchval()
row

'Tom'

In [68]:
row = crs.execute("SELECT COUNT(*) from TestA").fetchval()
row

6

In [69]:
conn.commit()

In [70]:
crs.close()

In [71]:
conn.close()

In [72]:
row = crs.execute("SELECT COUNT(*) from TestA").fetchval()
row

ProgrammingError: Attempt to use a closed cursor.

In [73]:
crs = conn.cursor()

ProgrammingError: Attempt to use a closed connection.

### Connecting to Bikestores Database

In [74]:
driver = '{ODBC Driver 17 for SQL Server}'
server = 'localhost'
database = 'BikeStores'
user = 'sa'
password = 'Ll123456'

In [76]:
def create_server_connection(driver, server, database, user, password):
    conn = None
    try:
        conn = pyodbc.connect(DRIVER = driver, SERVER = server, DATABASE = database, UID = user, PWD = password)
        print("MS SQL Server Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")
    return conn

In [77]:
conn = create_server_connection(driver, server, database, user, password)

MS SQL Server Database connection successful


In [78]:
crs = conn.cursor()

In [79]:
query = """SELECT TABLE_NAME
           FROM INFORMATION_SCHEMA.TABLES
           WHERE TABLE_TYPE='BASE TABLE'
        """

In [81]:
execute_query(conn, query)

Query successful!


In [82]:
crs.execute(query).fetchall()

[('categories', ),
 ('brands', ),
 ('products', ),
 ('customers', ),
 ('stores', ),
 ('staffs', ),
 ('orders', ),
 ('order_items', ),
 ('stocks', ),
 ('sysdiagrams', )]

In [83]:
crs.execute(query).fetchone()

('categories', )

In [84]:
crs.execute(query).fetchval()

'categories'

In [85]:
for row in crs.execute(query):
    print(row[0])

categories
brands
products
customers
stores
staffs
orders
order_items
stocks
sysdiagrams


In [86]:
for row in crs.execute(query):
    print(row.TABLE_NAME)

categories
brands
products
customers
stores
staffs
orders
order_items
stocks
sysdiagrams


In [87]:
for row in crs.execute(query):
    print(row)

('categories', )
('brands', )
('products', )
('customers', )
('stores', )
('staffs', )
('orders', )
('order_items', )
('stocks', )
('sysdiagrams', )


In [88]:
crs.execute("""select * from production.brands""").fetchall()

[(1, 'Electra'),
 (2, 'Haro'),
 (3, 'Heller'),
 (4, 'Pure Cycles'),
 (5, 'Ritchey'),
 (6, 'Strider'),
 (7, 'Sun Bicycles'),
 (8, 'Surly'),
 (9, 'Trek')]

In [89]:
pd.read_sql("""select * from production.brands""", con=conn)

,brand_id,brand_name
0,1,Electra
1,2,Haro
2,3,Heller
3,4,Pure Cycles
4,5,Ritchey
5,6,Strider
6,7,Sun Bicycles
7,8,Surly
8,9,Trek


In [91]:
df_products = pd.read_sql("""select * from production.products""", con=conn)
df_products

,product_id,product_name,brand_id,category_id,model_year,list_price
0,1,Trek 820 - 2016,9,6,2016,379.99
1,2,Ritchey Timberwolf Frameset - 2016,5,6,2016,749.99
2,3,Surly Wednesday Frameset - 2016,8,6,2016,999.99
3,4,Trek Fuel EX 8 29 - 2016,9,6,2016,2899.99
4,5,Heller Shagamaw Frame - 2016,3,6,2016,1320.99
...,...,...,...,...,...,...
316,317,Trek Checkpoint ALR 5 - 2019,9,7,2019,1999.99
317,318,Trek Checkpoint ALR 5 Women's - 2019,9,7,2019,1999.99
318,319,Trek Checkpoint SL 5 Women's - 2019,9,7,2019,2799.99
319,320,Trek Checkpoint SL 6 - 2019,9,7,2019,3799.99


In [92]:
import sqlalchemy
import urllib

In [93]:
driver = '{ODBC Driver 17 for SQL Server}'
server = 'localhost' 
database = 'TESTDB' 
user = 'sa' 
password = 'Ll123456'
conn = create_server_connection(driver, server, database, user, password)

MS SQL Server Database connection successful


In [94]:
params = 'DRIVER='+driver + ';SERVER='+server + ';PORT=1433;DATABASE=' + database + ';UID=' + user + ';PWD=' + password

In [95]:
db_params = urllib.parse.quote_plus(params)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))

In [96]:
df_products.to_sql("products", engine, index=False, if_exists="append", schema="dbo")

In [97]:
crs.close()
conn.close()